In [ ]:
import matplotlib.pyplot as pp
import numpy as np

import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
import tqdm


base_face = lambda: (
    pp.gca().add_patch(pp.Circle((0.5, 0.5), 0.5, fc=(0.9, 0.9, 0))),
    pp.gca().add_patch(pp.Circle((0.25, 0.6), 0.1, fc=(0, 0, 0))),
    pp.gca().add_patch(pp.Circle((0.75, 0.6), 0.1, fc=(0, 0, 0))),
)
patches = {
    "smile": lambda: pp.gca().add_patch(
        pp.Polygon(
            np.stack(
                [
                    np.linspace(0.2, 0.8),
                    0.3 - np.sin(np.linspace(0, 3.14)) * 0.15,
                ]
            ).T,
            closed=False,
            fill=False,
            lw=3,
        )
    ),
    "frown": lambda: pp.gca().add_patch(
        pp.Polygon(
            np.stack(
                [
                    np.linspace(0.2, 0.8),
                    0.15 + np.sin(np.linspace(0, 3.14)) * 0.15,
                ]
            ).T,
            closed=False,
            fill=False,
            lw=3,
        )
    ),
    "left_eb_down": lambda: pp.gca().add_line(
        pp.Line2D([0.15, 0.35], [0.75, 0.7], color=(0, 0, 0))
    ),
    "right_eb_down": lambda: pp.gca().add_line(
        pp.Line2D([0.65, 0.85], [0.7, 0.75], color=(0, 0, 0))
    ),
    "left_eb_up": lambda: pp.gca().add_line(
        pp.Line2D([0.15, 0.35], [0.7, 0.75], color=(0, 0, 0))
    ),
    "right_eb_up": lambda: pp.gca().add_line(
        pp.Line2D([0.65, 0.85], [0.75, 0.7], color=(0, 0, 0))
    ),
}
sorted_keys = sorted(patches.keys())


def draw_face(face):
    base_face()
    for i in face:
        patches[i]()
    pp.axis("scaled")
    pp.axis("off")


f, ax = pp.subplots(1, 2)
pp.sca(ax[0])
draw_face(["smile", "left_eb_down", "right_eb_down"])
pp.sca(ax[1])
draw_face(["frown", "left_eb_up", "right_eb_up"])

In [25]:
def has_overlap(face):
    # Can't have two overlapping eyebrows!
    if "left_eb_down" in face and "left_eb_up" in face:
        return True
    if "right_eb_down" in face and "right_eb_up" in face:
        return True
    # Can't have two overlapping mouths!
    if "smile" in face and "frown" in face:
        return True
    return False


def face_reward(face):
    if has_overlap(face):
        return 0
    eyebrows = "left_eb_down", "left_eb_up", "right_eb_down", "right_eb_up"
    # Must have exactly two eyebrows
    if sum([i in face for i in eyebrows]) != 2:
        return 0
    # We want twice as many happy faces as sad faces so here we give a reward of 2 for smiles
    if "smile" in face:
        return 4
    if "frown" in face:
        return 2  # and a reward of 1 for frowns
    # If we reach this point, there's no mouth
    return 0


def face_to_tensor(face):
    return torch.tensor([i in face for i in sorted_keys]).float()

In [26]:
import torch.nn as nn


class TBModel(nn.Module):
    def __init__(self, num_hid):
        super().__init__()
        # The input dimension is 6 for the 6 patches.
        self.mlp = nn.Sequential(
            nn.Linear(6, num_hid),
            nn.LeakyReLU(),
            # We now output 12 numbers, 6 for P_F and 6 for P_B
            nn.Linear(num_hid, 12),
        )
        # log Z is just a single number
        self.logZ = nn.Parameter(torch.ones(1))

    def forward(self, x):
        logits = self.mlp(x)
        # Slice the logits, and mask invalid actions (since we're predicting
        # log-values), we use -100 since exp(-100) is tiny, but we don't want -inf)
        P_F = logits[..., :6] * (1 - x) + x * -100
        P_B = logits[..., 6:] * x + (1 - x) * -100
        return P_F, P_B

In [ ]:
import time

# Instantiate model and optimizer
model = TBModel(512)
opt = torch.optim.Adam(model.parameters(), 3e-4)

# Let's keep track of the losses and the faces we sample
tb_losses = []
tb_sampled_faces = []
# To not complicate the code, I'll just accumulate losses here and take a
# gradient step every `update_freq` episode.
minibatch_loss = 0
update_freq = 2
ep_times = []

logZs = []
for episode in tqdm.tqdm(range(50000), ncols=40):
    # Each episode starts with an "empty state"
    state = []
    # Predict P_F, P_B
    P_F_s, P_B_s = model(face_to_tensor(state))
    total_P_F = 0
    total_P_B = 0
    start_time = time.time()

    for t in range(3):
        # Here P_F is logits, so we want the Categorical to compute the softmax for us
        cat = Categorical(logits=P_F_s)
        action = cat.sample()
        # "Go" to the next state
        new_state = state + [sorted_keys[action]]
        # Accumulate the P_F sum
        total_P_F += cat.log_prob(action)

        if t == 2:
            # If we've built a complete face, we're done, so the reward is > 0
            # (unless the face is invalid)
            reward = torch.tensor(face_reward(new_state)).float()
        # We recompute P_F and P_B for new_state
        P_F_s, P_B_s = model(face_to_tensor(new_state))
        # Here we accumulate P_B, going backwards from `new_state`. We're also just
        # going to use opposite semantics for the backward policy. I.e., for P_F action
        # `i` just added the face part `i`, for P_B we'll assume action `i` removes
        # face part `i`, this way we can just keep the same indices.
        total_P_B += Categorical(logits=P_B_s).log_prob(action)

        # Continue iterating
        state = new_state

    ep_times.append(time.time() - start_time)
    # We're done with the trajectory, let's compute its loss. Since the reward can
    # sometimes be zero, instead of log(0) we'll clip the log-reward to -20.
    loss = (
        model.logZ + total_P_F - torch.log(reward).clip(-20) - total_P_B
    ).pow(2)
    minibatch_loss += loss

    # Add the face to the list, and if we are at an
    # update episode, take a gradient step.
    tb_sampled_faces.append(state)
    if episode % update_freq == 0:
        tb_losses.append(minibatch_loss.item())
        minibatch_loss.backward()
        opt.step()
        opt.zero_grad()
        minibatch_loss = 0
        logZs.append(model.logZ.item())

In [ ]:
model

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
for p in model_parameters:
    print(p.size())
params = sum([np.prod(p.size()) for p in model_parameters])
params

In [44]:
for p in model_parameters:
    print(p.size())

In [ ]:
# print total number of parameters in the model
print(sum(p.numel() for p in model.parameters()))

In [ ]:
np.mean(np.exp(logZs))

In [ ]:
# take every 5th loss value
losses = tb_losses[::5]
zs = logZs[::5]

f, ax = pp.subplots(2, 1, figsize=(10, 6))
pp.sca(ax[0])
pp.plot(losses)
pp.yscale("log")
pp.ylabel("loss")
pp.sca(ax[1])
pp.plot(np.exp(zs))
pp.ylabel("estimated Z")

# save figure
pp.savefig("./classical-training-loss-smile.png")

In [ ]:
print("Average episode time: ", np.mean(ep_times), "s")

In [ ]:
f, ax = pp.subplots(8, 8, figsize=(4, 4))
print(
    "Ratio of faces with a smile:",
    sum(["smile" in i for i in tb_sampled_faces[-100:]]) / 100,
)
print(
    "Ratio of valid faces:",
    sum([face_reward(i) > 0 for i in tb_sampled_faces[-100:]]) / 100,
)
for i, face in enumerate(tb_sampled_faces[-64:]):
    pp.sca(ax[i // 8, i % 8])
    draw_face(face)